In [1]:
# ALWAYS RUN THIS CELL
# from AntennaNetwork import AntennaCNN#imports for next cells 
import numpy as np
import h5py
import torch
from tqdm.auto import tqdm
import torch.nn as nn
from AntennaNetwork import AntennaCNN

model = AntennaCNN()
# print(model.fc_1.bias.shape)
# weights of fully conneceted layer are stored in transposee for some reeason, but need to keeep this in mind

# for name, param in model.named_parameters():
#     print(name, param.size())


In [2]:
# RUN THIS CELL TO IMPORT MODEL PARAMS FROM MATLAB (not necessary if already run)

# Load matlab data as file
f = h5py.File('antenna_test.mat','r')

# C is a matlab variable cell array storing the weights and biases of conv and forward layers
weights_and_biases = f.get('C')

# load batchnorm params from matlab file
bn_params = f.get("BN_Params") 

# keys that can be accessed (to check that the matlab variables are loaded correctly
# print(f.keys())

# This works for getting the data into python. Have to transpose the last two dimensions for some reason for 
# them to match it seems
"""
data = f.get('XTest')
data = np.array(data) # For converting to a NumPy arrayo
data = np.transpose(data, [0, 1, 3, 2])

print(data[0,:,:,:])
"""


# stores reference to i'th item in C, then use to get from file

# params.append(torch.tensor(f[weights[i,0]]))

# This loop sets up the convolutional layer parameters of the model, weights and biases
i = 0
for name, params in list(model.named_parameters()):
    if (i % 4) != 0 and (i % 4) != 1:
        i+=1
        continue
    if i//4 == 10:
        break

    # Set convolutional layer params
    if i % 4 == 0:
        layers_weights = torch.tensor(np.array(f[weights_and_biases[(i//4)*2,0]]))
        # try with and without transposing maybe???
        layers_weights = torch.transpose(layers_weights, 3, 2)
    else:
        layers_weights = torch.squeeze(torch.tensor(np.array(f[weights_and_biases[(i//4)*2+1,0]])))

    with torch.no_grad():
        layers_weights = layers_weights
        params.data = nn.parameter.Parameter(layers_weights)
    i+=1

# torch.save(model.state_dict(), "./saved/AntennaCNN")
# model.load_state_dict(torch.load("./saved/AntennaCNN"))

# This loop sets up the fully connected layer parameters of the model, weights and biases
i = 0
for name, params in list(model.named_parameters()):
    if i < 40 or ((i % 4) != 0 and (i % 4) != 1):
        i+=1
        continue
    # print(name, i)

    # Set forward layer params
    if i % 4 == 0:
        # transpose because torch Linear stores weights this way 
        layers_weights = torch.tensor(np.array(f[weights_and_biases[(i//4)*2,0]])).T
        # print(layers_weights.shape)
    else:
        # biases
        layers_weights = torch.squeeze(torch.tensor(np.array(f[weights_and_biases[(i//4)*2+1,0]])))

    with torch.no_grad():
        # print(layers_weights.shape)
        layers_weights = layers_weights
        params.data = nn.parameter.Parameter(layers_weights)

    i+=1

# torch.save(model.state_dict(), "./saved/AntennaCNN")

# This loop sets up the batch normalization layer parameters
i = 0
for name, m in model.named_children():
    if 'batchnorm' not in name:
        continue
    # mean = self.running_mean
    # variance = self.running_var
    # gamma = self.weight
    # beta = self.bias
    # batchnorm_params[i] will be four separate arrays, in the order:
    # running_mean, running_var, weight/gamme, beta/bias
    running_mean = torch.tensor(np.array(f[f[bn_params[i,0]][0,0]])).squeeze()#.double()
    running_var = torch.tensor(np.array(f[f[bn_params[i,0]][0,1]])).squeeze()#.double()
    weight = torch.tensor(np.array(f[f[bn_params[i,0]][0,2]])).squeeze()#.double()
    bias = torch.tensor(np.array(f[f[bn_params[i,0]][0,3]])).squeeze()#.double()

    # # Set convolutional layer params
    # layers_weights = torch.tensor(np.array(f[batchnorm_params[i]]))

    with torch.no_grad():
        m.running_mean = nn.parameter.Parameter(running_mean, requires_grad=False)
        m.running_var = nn.parameter.Parameter(running_var, requires_grad=False)
        m.weight = nn.parameter.Parameter(weight)
        m.bias = nn.parameter.Parameter(bias)

    i+=1

torch.save(model.state_dict(), "./saved/AntennaCNN")

In [3]:
# RUN TO LOAD IN MODEL BEFORE MAKING PREDICTIONS
model.load_state_dict(torch.load("./saved/AntennaCNN"))
model.conv_1.weight.shape
model.conv_1.weight[0,0,:,:]


tensor([[ 1.1965e-02, -2.8258e-02,  1.9644e-01,  1.6783e-02,  1.6049e-02,
         -6.9570e-02, -2.8445e-02, -3.2110e-02],
        [ 2.2386e-02, -8.9089e-03,  1.2918e-01,  1.5090e-01, -1.4836e-01,
         -8.6447e-02,  3.4734e-02, -1.3562e-03],
        [ 5.7421e-02,  9.3961e-02, -2.6923e-02,  1.4443e-01,  5.9091e-02,
         -9.2035e-02, -7.2029e-03, -1.1172e-02],
        [ 3.6873e-02,  4.4210e-02,  1.9583e-01,  2.5301e-01, -4.0367e-02,
         -6.0448e-02,  3.9607e-02,  1.9913e-02],
        [-4.5293e-02,  4.6942e-02,  1.1650e-01,  1.4515e-01,  2.2966e-01,
          7.1594e-02,  5.0623e-03, -3.8013e-02],
        [-8.8023e-02, -4.4259e-03, -1.4503e-01, -1.2499e-01,  4.0259e-02,
          1.0838e-01,  3.3600e-02,  6.6949e-02],
        [-3.0182e-02, -3.1303e-02, -2.5854e-02, -1.0193e-02,  4.4852e-02,
         -2.6549e-02,  6.9079e-02,  4.6409e-02],
        [ 7.6728e-05, -8.3340e-02,  6.0726e-03,  4.8939e-02,  2.0225e-02,
         -5.2344e-02, -4.5712e-02, -1.4330e-02]], grad_fn=<SliceB

In [1]:
## RUN THIS CELL TO LOAD MODEL IN FROM STATE DICT, RUN TESTS
import matplotlib.pyplot as plt

# model.load_state_dict(torch.load("./saved/AntennaCNN"))
# params = model.state_dict()

# testing to make sure these match matlab weights
# convweights[outut_chan, input_chan, x, y] should match matlabs [x, y, input_chan, output_chan]
# output_chan = 37
# input_chan = 57
# print(params["conv_5.weight"].size())
# print(params["conv_5.weight"][output_chan, input_chan, :, :])
# Seems to be working correctly!!

# after inspecting, this looks good. Seems to load the correct model from matlab
model.load_state_dict(torch.load("./saved/AntennaCNN"))
model.eval()
# print(model.fc_1.weight.shape) # should be 2000 x 13824 sinc estored in transpose

f = h5py.File('antenna_test.mat','r')
    
x_test = torch.transpose(torch.tensor(np.array(f["XTest"])), 2, 3)
y_test = torch.tensor(np.array(f["YTest"])).T
# print(x_test.shape)
# print(y_test.shape)

def plot_example(number, x, y):

    # image seems transposed coming in from matlab
    test_struct = x[number,0,:,:]
    test_res = y[number,:]

    # now also get model predictions for this structure
    with torch.no_grad():
        to_pass = test_struct.unsqueeze(0).unsqueeze(0).float()
        # normalize the input
        to_pass = to_pass - 0.6667
        model_res = model(to_pass)
    
        plt.subplot(1,3,1)
        plt.imshow(test_struct, cmap="gray")
        plt.title("Test struct #: {}".format(number+1))
        plt.subplot(1,3,2)
        x_ax = [i for i in range(len(test_res))]
        plt.plot(x_ax, test_res, label="true spectrum")
        plt.plot(x_ax, model_res.squeeze(), label = "pred spectrum")
        plt.title("Test Data #: {}".format(number+1))
        plt.show()

plot_example(9, x_test, y_test)
plot_example(49, x_test, y_test)
plot_example(59, x_test, y_test)
# print(x_test[9,:,0,0])


NameError: name 'model' is not defined

In [13]:
x = torch.tensor([[[1, 3], 
                   [2, 4]], 
                  [[5, 7], 
                   [6, 8]], 
                  [[9, 11], 
                   [10, 12]]])
print(x.shape)
x = x.permute(0,2,1)
print(x.shape)
print(x)
print(x.flatten())

torch.Size([3, 2, 2])
torch.Size([3, 2, 2])
tensor([[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]],

        [[ 9, 10],
         [11, 12]]])
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])


In [39]:
from pathlib import Path
import os

# Loading in 300nm (or is it micron) data
# DATA_PATH = os.path.join(Path(os.getcwd()).parent, "Test_Results300")

# files = os.listdir(DATA_PATH)
# for i in range(0, len(files)-1, 2):
#     data = os.path.join(DATA_PATH, files[i])
#     struct = os.path.join(DATA_PATH, files[i+1])

#     if os.path.isfile(f) and os.path.isfile(f2):
        
        

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_23801_data.s8p
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_23801_struct.txt
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_33615_data.s8p
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_33615_struct.txt
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_13667_data.s8p
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_13667_struct.txt
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_23802_data.s8p
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_23802_struct.txt
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_33616_data.s8p
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_33616_struct.txt
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_13668_data.s8p
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Results300/trial_13668_struct.txt
/home/sbfisher/Cadence/Thesis/PythonCode/Test_Result

In [ ]:
# save dataset as csv:
